<a href="https://colab.research.google.com/github/LeyiYe/IMLO-assessment/blob/main/final_IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import Flowers102
from torch.utils.data import DataLoader
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

# Define transforms for data augmentation and normalization
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.RandomResizedCrop(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset with train/val/test splits
train_dataset = Flowers102(root='./data', split='train', transform=train_transform, download=True)
val_dataset = Flowers102(root='./data', split='val', transform=val_transform, download=True)

test_dataset = Flowers102(root='./data', split='test', transform=val_transform, download=True)

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


class CNN_NN(nn.Module):
    def __init__(self, num_classes=102):
        super(CNN_NN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv5= nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv6= nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        #self.conv7= nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        #self.conv8= nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation =1, ceil_mode=False)

        # Initialize the size of the fully connected layer based on the sample input
        self._to_linear = None
        self.convs = nn.Sequential(
            self.conv1,
            nn.ReLU(inplace=True),
            #self.pool,
            self.conv2,
            nn.ReLU(inplace=True),
            self.pool,
            self.conv3,
            nn.ReLU(inplace=True),
            #self.pool,
            self.conv4,
            nn.ReLU(inplace=True),
            self.pool,
            self.conv5,
            nn.ReLU(inplace=True),
            #self.pool
            #self.conv6,
            #nn.ReLU(inplace=True),
            self.conv6,
            nn.ReLU(inplace=True),
            self.pool,
           # self.conv7,
            #nn.ReLU(inplace=True),
            #self.conv8,
            #nn.ReLU(inplace=True),
            #self.conv8,
           # nn.ReLU(inplace=True),
           # self.pool,
            #self.conv8,
           # nn.ReLU(inplace=True),
           # self.conv8,
           # nn.ReLU(inplace=True),
           # self.conv8,
           # nn.ReLU(inplace=True),
           # self.pool,

        )
        self._initialize_linear_layer()

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def _initialize_linear_layer(self):
        # Create a dummy input tensor to calculate the output size after conv layers
        x = torch.randn(1, 3, 224, 224)
        x = self.convs(x)
        self._to_linear = x.view(1, -1).shape[1]

    def forward(self, x):
        x = self.convs(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor while preserving the batch size
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Define the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_classes = 102  # Number of output classes
learning_rate = 0.001
num_epochs = 1200

# Initialize the model
model = CNN_NN(num_classes=num_classes).to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy
    val_loss = val_loss / len(val_loader.dataset)
    accuracy = correct / total

    model.eval()  # Set the model to evaluation mode

    test_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad():  # No need to compute gradients for evaluation
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    # Calculate average test loss and accuracy
    test_loss = test_loss / len(test_loader.dataset)
    test_accuracy = test_correct / test_total


    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Training Loss: {epoch_loss:.4f}, '
          f'Validation Loss: {val_loss:.4f}, '
          f'Validation Percentage:{accuracy*100:.4f}%,'
          f'Test Loss: {test_loss:.4f}, '
          f'Test Percentage:{test_accuracy*100:.4f}%'
          )

print('Finished Training')

Epoch [1/1200], Training Loss: 4.6455, Validation Loss: 4.6203, Validation Percentage:0.9804%,Test Loss: 4.6209, Test Percentage:0.7806%
Epoch [2/1200], Training Loss: 4.6098, Validation Loss: 4.4841, Validation Percentage:1.5686%,Test Loss: 4.4700, Test Percentage:2.1955%
Epoch [3/1200], Training Loss: 4.5115, Validation Loss: 4.4359, Validation Percentage:1.5686%,Test Loss: 4.4407, Test Percentage:2.0003%
Epoch [4/1200], Training Loss: 4.4665, Validation Loss: 4.4457, Validation Percentage:1.6667%,Test Loss: 4.4664, Test Percentage:2.6508%
Epoch [5/1200], Training Loss: 4.4443, Validation Loss: 4.3806, Validation Percentage:2.3529%,Test Loss: 4.4529, Test Percentage:1.4474%
Epoch [6/1200], Training Loss: 4.3657, Validation Loss: 4.2995, Validation Percentage:2.8431%,Test Loss: 4.3489, Test Percentage:1.8052%
Epoch [7/1200], Training Loss: 4.3145, Validation Loss: 4.2433, Validation Percentage:3.7255%,Test Loss: 4.3210, Test Percentage:2.2443%
Epoch [8/1200], Training Loss: 4.2678, Va

In [ ]:
test_loss = 0.0
test_correct = 0
test_total = 0
#Test
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

# Calculate average test loss and accuracy
test_loss = test_loss / len(test_loader.dataset)
test_accuracy = test_correct / test_total


print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Training Loss: {epoch_loss:.4f}, '
          f'Test Loss: {test_loss:.4f}, '
          f'Test Percentage:{test_accuracy*100:.4f}%'
          )

Epoch [1200/1200], Training Loss: 0.8737, Test Loss: 3.3277, Test Percentage:40.1203%


In [11]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import Flowers102
from torch.utils.data import DataLoader
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

# Define transforms for data augmentation and normalization
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.RandomResizedCrop(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset with train/val/test splits
train_dataset = Flowers102(root='./data', split='train', transform=train_transform, download=True)
val_dataset = Flowers102(root='./data', split='val', transform=val_transform, download=True)

test_dataset = Flowers102(root='./data', split='test', transform=val_transform, download=True)

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


class CNN_NN(nn.Module):
    def __init__(self, num_classes=102):
        super(CNN_NN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv5= nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv6= nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation =1, ceil_mode=False)

        # Initialize the size of the fully connected layer based on the sample input
        self._to_linear = None
        self.convs = nn.Sequential(
            self.conv1,
            nn.ReLU(inplace=True),
            self.conv2,
            nn.ReLU(inplace=True),
            self.pool,
            self.conv3,
            nn.ReLU(inplace=True),
            self.conv4,
            nn.ReLU(inplace=True),
            self.pool,
            self.conv5,
            nn.ReLU(inplace=True),
            self.conv6,
            nn.ReLU(inplace=True),
            self.pool,
        )
        self._initialize_linear_layer()

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def _initialize_linear_layer(self):
        # Create a dummy input tensor to calculate the output size after conv layers
        x = torch.randn(1, 3, 224, 224)
        x = self.convs(x)
        self._to_linear = x.view(1, -1).shape[1]

    def forward(self, x):
        x = self.convs(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor while preserving the batch size
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Define the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_classes = 102  # Number of output classes
learning_rate = 0.001
num_epochs = 1200

# Initialize the model
model = CNN_NN(num_classes=num_classes).to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy
    val_loss = val_loss / len(val_loader.dataset)
    accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Training Loss: {epoch_loss:.4f}, '
          f'Validation Loss: {val_loss:.4f}, '
          f'Validation Percentage:{accuracy*100:.4f}%,'
          )

print('Finished Training')

Epoch [1/1200], Training Loss: 4.6457, Validation Loss: 4.6176, Validation Percentage:1.6667%,
Epoch [2/1200], Training Loss: 4.6010, Validation Loss: 4.4682, Validation Percentage:1.1765%,
Epoch [3/1200], Training Loss: 4.5066, Validation Loss: 4.4327, Validation Percentage:1.5686%,
Epoch [4/1200], Training Loss: 4.4606, Validation Loss: 4.4088, Validation Percentage:1.8627%,
Epoch [5/1200], Training Loss: 4.4516, Validation Loss: 4.4128, Validation Percentage:2.3529%,
Epoch [6/1200], Training Loss: 4.4306, Validation Loss: 4.4106, Validation Percentage:1.7647%,
Epoch [7/1200], Training Loss: 4.3816, Validation Loss: 4.3780, Validation Percentage:2.2549%,
Epoch [8/1200], Training Loss: 4.3419, Validation Loss: 4.2687, Validation Percentage:2.3529%,
Epoch [9/1200], Training Loss: 4.3052, Validation Loss: 4.2243, Validation Percentage:2.8431%,
Epoch [10/1200], Training Loss: 4.2819, Validation Loss: 4.1866, Validation Percentage:3.5294%,
Epoch [11/1200], Training Loss: 4.2200, Validatio

In [12]:
test_loss = 0.0
test_correct = 0
test_total = 0
#Test
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

# Calculate average test loss and accuracy
test_loss = test_loss / len(test_loader.dataset)
test_accuracy = test_correct / test_total


print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Training Loss: {epoch_loss:.4f}, '
          f'Test Loss: {test_loss:.4f}, '
          f'Test Percentage:{test_accuracy*100:.4f}%'
          )

Epoch [1200/1200], Training Loss: 0.8494, Test Loss: 3.2561, Test Percentage:43.1127%
